In [1]:
from sqlalchemy import create_engine,text

In [2]:
from sqlalchemy.engine import URL

In [5]:
import pandas as pd

In [6]:
conn_str = URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="Nanu@2903",
    host = "localhost",
    port = 3306,
    database="employees"
)

In [7]:
engine = create_engine(conn_str)

In [8]:
employees= engine.connect()

In [9]:
pd.read_sql("show tables",employees)

,Tables_in_employees
0,current_dept_emp
1,departments
2,dept_emp
3,dept_emp_latest_date
4,dept_manager
5,employees
6,salaries
7,titles


In [107]:
dept_emp_de = pd.read_sql("select * from dept_emp",employees)          #(emp_no, dept_no, from_date, to_date)4
departments_d = pd.read_sql("select * from departments",employees)        #(dept_no, dept_name)
employees_e = pd.read_sql("select * from employees",employees)          #(emp_no, first_name, last_name, gender, birth_date)
dept_manager_dm = pd.read_sql("select * from dept_manager",employees)      #(emp_no, dept_no, from_date, to_date)
salaries_s = pd.read_sql("select * from salaries",employees)           #(emp_no, salary, from_date, to_date)
titles_t = pd.read_sql("select * from titles",employees)             #(emp_no, title, from_date, to_date)

In [23]:
#Q.1 List all employees along with their department names.

In [24]:
emp_dept = pd.merge(employees_e, dept_emp_de, on="emp_no")
emp_dept_names = pd.merge(emp_dept, departments_d, on="dept_no")

final_result = emp_dept_names[["emp_no", "first_name", "last_name", "dept_name"]]
print(final_result)

        emp_no first_name last_name        dept_name
0        10001     Georgi   Facello      Development
1        10002    Bezalel    Simmel            Sales
2        10003      Parto   Bamford       Production
3        10004  Chirstian   Koblick       Production
4        10005    Kyoichi  Maliniak  Human Resources
...        ...        ...       ...              ...
331598  499995     Dekang  Lichtner       Production
331599  499996       Zito      Baaz       Production
331600  499997    Berhard    Lenart      Development
331601  499998   Patricia   Breugel          Finance
331602  499999     Sachin   Tsukuda       Production

[331603 rows x 4 columns]


In [26]:
#Q.2 Find the total number of employees in each department.

In [47]:
a = pd.merge(dept_emp_de, departments_d, on="dept_no")
emp_count = a.groupby("dept_name")["emp_no"].count().reset_index()

emp_count.rename(columns={"emp_no": "total_emps"}, inplace=True)
print(emp_count)

            dept_name  total_emps
0    Customer Service       23580
1         Development       85707
2             Finance       17346
3     Human Resources       17786
4           Marketing       20211
5          Production       73485
6  Quality Management       20117
7            Research       21126
8               Sales       52245


In [ ]:
#Q.3 Get the details of employees who are currently working in a department (i.e., to_date = '9999-01-01').

In [59]:
curr_emp = pd.merge(dept_emp_de, employees_e, on="emp_no")
curr_emp = pd.merge(curr_emp, departments_d, on="dept_no")
curr_dept_emp = dept_emp_de[dept_emp_de["to_date"] == ("9999-01-01")]
result = curr_emp[["emp_no", "first_name", "last_name", "gender", "dept_name","to_date"]]
print(result)

        emp_no first_name last_name gender        dept_name     to_date
0        10001     Georgi   Facello      M      Development  9999-01-01
1        10002    Bezalel    Simmel      F            Sales  9999-01-01
2        10003      Parto   Bamford      M       Production  9999-01-01
3        10004  Chirstian   Koblick      M       Production  9999-01-01
4        10005    Kyoichi  Maliniak      M  Human Resources  9999-01-01
...        ...        ...       ...    ...              ...         ...
331598  499995     Dekang  Lichtner      F       Production  9999-01-01
331599  499996       Zito      Baaz      M       Production  9999-01-01
331600  499997    Berhard    Lenart      M      Development  9999-01-01
331601  499998   Patricia   Breugel      M          Finance  9999-01-01
331602  499999     Sachin   Tsukuda      M       Production  9999-01-01

[331603 rows x 6 columns]


In [ ]:
#Q.4 Calculate average salary of employees department-wise.

In [45]:
emp_salary = pd.merge(dept_emp_de, salaries_s, on="emp_no")
emp_sal_dept = pd.merge(emp_salary, departments_d, on="dept_no")
avg_sal_by_dept = emp_sal_dept.groupby("dept_name")["salary"].mean().reset_index()
avg_sal_by_dept.columns = ["Department", "Average_Salary"]
print(avg_sal_by_dept)

           Department  Average_Salary
0    Customer Service    58755.440627
1         Development    59503.574995
2             Finance    70159.466220
3     Human Resources    55353.520326
4           Marketing    71901.723674
5          Production    59539.789908
6  Quality Management    57294.657039
7            Research    59866.243527
8               Sales    80776.620397


In [ ]:
#Q.5 List employees with their current job title and current salary.

In [51]:
curr_salary = salaries_s[salaries_s["to_date"]=="9999-01-01"]
curr_title  = titles_t[titles_t["to_date"]=="9999-01-01"]
emp_salary = pd.merge(employees_e, salaries_s, on = "emp_no")
emp_salary_title = pd.merge(emp_salary, titles_t, on = "emp_no")
result = emp_salary_title[["emp_no", "first_name", "last_name", "title", "salary"]]
print(result)

         emp_no first_name last_name            title  salary
0         10001     Georgi   Facello  Senior Engineer   60117
1         10001     Georgi   Facello  Senior Engineer   62102
2         10001     Georgi   Facello  Senior Engineer   66074
3         10001     Georgi   Facello  Senior Engineer   66596
4         10001     Georgi   Facello  Senior Engineer   66961
...         ...        ...       ...              ...     ...
1580439  201771   Fumitaka   Gammage  Senior Engineer   60056
1580440  201771   Fumitaka   Gammage         Engineer   62899
1580441  201771   Fumitaka   Gammage  Senior Engineer   62899
1580442  201771   Fumitaka   Gammage         Engineer   66792
1580443  201771   Fumitaka   Gammage  Senior Engineer   66792

[1580444 rows x 5 columns]


In [ ]:
#Q.6 Find employees who have worked in more than one department.

In [75]:
dept_count = dept_emp_de.groupby("emp_no")["dept_no"].nunique().reset_index()
multi_dept_emps = dept_count[dept_count["dept_no"] > 1]
multi_dept_emps2 = pd.merge(multi_dept_emps, employees_e, on="emp_no", how="inner")
result = multi_dept_emps2[["emp_no", "first_name", "last_name"]]
print(result)

       emp_no    first_name last_name
0       10010     Duangkaew  Piveteau
1       10018      Kazuhide      Peha
2       10029         Otmar    Herbst
3       10040         Weiyi   Meriste
4       10050       Yinghua    Dredge
...       ...           ...       ...
31574  499955  Kankanahalli    Zucker
31575  499964         Randy    Matzov
31576  499975        Masali   Chorvat
31577  499983           Uri    Juneja
31578  499992        Siamak  Salverda

[31579 rows x 3 columns]


In [ ]:
#Q.7 Get all female employees working in the 'Sales' department.


In [89]:
emp_dept = pd.merge(employees_e, dept_emp_de, on="emp_no")
emp_dept2 = pd.merge(emp_dept, departments_d, on="dept_no")
female_emp = emp_dept2[(emp_dept2["gender"]=="F") & (emp_dept2["dept_name"]=="Sales")]
result = female_emp[["emp_no", "first_name", "last_name", "gender", "dept_name"]]
print(result)

        emp_no first_name last_name gender dept_name
1        10002    Bezalel    Simmel      F     Sales
44       10041        Uri    Lenart      F     Sales
57       10053     Sanjiv  Zschoche      F     Sales
94       10087    Xinglin   Eugenio      F     Sales
95       10088   Jungsoon  Syrzycki      F     Sales
...        ...        ...       ...    ...       ...
331559  499960     Gaetan  Veldwijk      F     Sales
331566  499966    Mihalis  Crabtree      F     Sales
331588  499986     Nathan     Ranta      F     Sales
331589  499987      Rimli    Dusink      F     Sales
331590  499988   Bangqing   Kleiser      F     Sales

[20854 rows x 5 columns]


In [ ]:
#Q.8 Find the oldest employee in the company.

In [135]:
employees_e['birth_date'] = pd.to_datetime(employees_e['birth_date'])

oldest_employee = employees_e.loc[employees_e['birth_date'].idxmin()]

print("Oldest Employee Details:")
print(f"Emp No     : {oldest_employee['emp_no']}")
print(f"Name       : {oldest_employee['first_name']} {oldest_employee['last_name']}")
print(f"Gender     : {oldest_employee['gender']}")
print(f"Birth Date : {oldest_employee['birth_date'].date()}")

Oldest Employee Details:
Emp No     : 65308
Name       : Jouni Pocchiola
Gender     : M
Birth Date : 1952-02-01


In [ ]:
#Q.9 Sort employees by highest salary.

In [95]:
emp_salary = pd.merge(employees_e, salaries_s, on="emp_no")
highest_salary = emp_salary.sort_values(by="salary", ascending=False)
print(highest_salary)

        emp_no  birth_date  first_name  last_name gender   hire_date  salary  \
318728   43624  1953-11-14    Tokuyasu      Pesch      M  1985-03-26  158220   
318727   43624  1953-11-14    Tokuyasu      Pesch      M  1985-03-26  157821   
360032   47978  1956-03-24      Xiahua   Whitcomb      M  1985-07-18  155709   
943635  109334  1955-08-02     Tsutomu  Alameldin      M  1985-02-15  155377   
943637  109334  1955-08-02     Tsutomu  Alameldin      M  1985-02-15  155190   
...        ...         ...         ...        ...    ...         ...     ...   
234513   34707  1955-10-26  Jacqueline      Syang      M  1985-07-23   38851   
383554   50419  1954-08-11       Vishu      Biran      F  1993-09-23   38850   
514589   64198  1960-01-04   Mechthild   Langford      M  1985-10-21   38836   
55309    15830  1963-10-22       Yurij   Narwekar      M  1993-09-14   38812   
372370   49239  1962-01-18      Fumiya       Unno      M  1992-09-18   38735   

         from_date     to_date  
318728

In [ ]:
#Q.10 List departments which currently don't have a manager assigned.

In [107]:
curr_manager = dept_manager_dm[dept_manager_dm["to_date"] == "9999-01-01"]
merge = pd.merge(curr_manager, departments_d, on="dept_no", how="right")
no_manager = merge[merge["emp_no"].isna()]
result = no_manager[["dept_no", "dept_name"]]
print(result)

  dept_no           dept_name
0    d009    Customer Service
1    d005         Development
2    d002             Finance
3    d003     Human Resources
4    d001           Marketing
5    d004          Production
6    d006  Quality Management
7    d008            Research
8    d007               Sales


In [4]:
#Q.11 Trend of average salary year-wise across the company.

In [132]:
salaries_s["year"] = pd.to_datetime(salaries_s["from_date"]).dt.year
avg_salary_by_year = salaries_s.groupby("year")["salary"].mean().reset_index()
print(avg_salary_by_year)

    year        salary
0   1985  53419.776250
1   1986  54303.321742
2   1987  55036.680852
3   1988  55865.363190
4   1989  56781.014351
5   1990  57783.804562
6   1991  58742.613292
7   1992  59679.225045
8   1993  60656.125052
9   1994  61643.287093
10  1995  62584.015074
11  1996  63547.501804
12  1997  64478.140581
13  1998  65494.420380
14  1999  66501.251952
15  2000  68515.573929
16  2001  70647.821233
17  2002  72622.588354


In [ ]:
#Q.12 Find employees who changed titles over time.

In [134]:
titles_t["from_date"] = pd.to_datetime(titles_t["from_date"])
titles_t = titles_t.sort_values(by=["emp_no", "from_date"])
title_change = titles_t.groupby("emp_no")["title"].nunique().reset_index()
title_change_emp = title_change[title_change["title"] > 1]
result = pd.merge(title_change_emp, employees_e, on='emp_no', how='inner')
result = result[["emp_no", "first_name", "last_name", "title"]]
print(result)

        emp_no first_name  last_name  title
0        10004  Chirstian    Koblick      2
1        10005    Kyoichi   Maliniak      2
2        10007    Tzvetan  Zielinski      2
3        10009     Sumant       Peac      3
4        10012   Patricio  Bridgland      2
...        ...        ...        ...    ...
140265  499988   Bangqing    Kleiser      2
140266  499992     Siamak   Salverda      2
140267  499996       Zito       Baaz      2
140268  499997    Berhard     Lenart      2
140269  499998   Patricia    Breugel      2

[140270 rows x 4 columns]


In [ ]:
#Q.13 Compare average salary of male vs female employees for each department.

In [33]:
emp_sal = pd.merge(employees_e, salaries_s, on= "emp_no")
dept_sal = pd.merge(emp_sal, dept_emp_de, on="emp_no")
emp_dept = pd.merge(dept_sal, departments_d, on="dept_no")

avg_salary = emp_dept.groupby(["dept_name", "gender"])["salary"].mean()
print(avg_salary)

dept_name           gender
Customer Service    F         58998.725316
                    M         58590.990909
Development         F         59391.952061
                    M         59576.334350
Finance             F         69914.920151
                    M         70327.028019
Human Resources     F         55596.374912
                    M         55196.550297
Marketing           F         71464.475047
                    M         72198.192617
Production          F         59455.999051
                    M         59596.357507
Quality Management  F         57423.311558
                    M         57206.895155
Research            F         59712.779945
                    M         59965.769015
Sales               F         80626.556859
                    M         80879.758464
Name: salary, dtype: float64


In [ ]:
#Q.14 Find departments where average salary is above overall company average.

In [51]:
emp_salary =  pd.merge(dept_emp_de, salaries_s, on="emp_no")
dept_emp_salary = pd.merge(emp_salary, departments_d, on="dept_no")

overall_salary = dept_emp_salary["salary"].mean()

dept_avg_salary = dept_emp_salary.groupby("dept_name")["salary"].mean()

above_avg_depts = dept_avg_salary[dept_avg_salary > overall_salary]

print("overall average salary:",overall_salary)
print("\nDepartments with above average salary:")
print(above_avg_depts)

overall average salary: 63771.92011442427

Departments with above average salary:
dept_name
Finance      70159.466220
Marketing    71901.723674
Sales        80776.620397
Name: salary, dtype: float64


In [131]:
#Q.15 Top 5 employees with the most job title changes.

In [133]:
emp_title = pd.merge(employees_e, titles_t, on="emp_no")

title_change = emp_title.groupby("first_name")["title"].nunique().sort_values(ascending=False)

top5 =  title_change.head(5)
print(top5)

first_name
Dung           7
Leon           7
Arie           7
Przemyslawa    7
Isamu          7
Name: title, dtype: int64
